<a href="https://colab.research.google.com/github/b3yotch/Creative_upaay/blob/main/Creative_Upaay_semantic_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install sentence-transformers faiss-cpu langchain llama-index nltk PyPDF2 requests bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 45.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 63.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20

In [2]:
import os
import nltk
import requests
from bs4 import BeautifulSoup
import PyPDF2
from io import BytesIO
from sentence_transformers import SentenceTransformer
import faiss
from langchain.text_splitter import RecursiveCharacterTextSplitter
import numpy as np

In [3]:
def load_documents(sources):
    """
    Load documents from  sources ( web URLs).

    sources: List of dictionaries with 'path' and 'type' keys
    """
    documents = []
    for source in sources:
        path = source['path']
        source_type = source['type']

        try:

            if source_type == 'url':
                content = extract_text_from_url(path)
                documents.append({"content": content, "source": path})

        except Exception as e:
            print(f"Error loading {path}: {e}")

    return documents

In [4]:
def extract_text_from_url(url):
    """Extract text and title from a web page."""
    text = ""
    title = "Untitled"

    try:
        response = requests.get(url)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text, 'html.parser')

            # Get the title tag or fallback to <h1> or other header
            if soup.title and soup.title.string:
                title = soup.title.string.strip()
            else:
                h1 = soup.find('h1')
                if h1:
                    title = h1.get_text(strip=True)

            # Extract article or body text
            article = soup.find('article') or soup.find('body')
            if article:
                for script in article(["script", "style"]):
                    script.extract()
                text = article.get_text(separator="\n")
            else:
                text = soup.get_text(separator="\n")

    except Exception as e:
        print(f"Error extracting from URL {url}: {e}")

    return title, text


In [5]:
def process_sources():
    """Process sources for the knowledge base."""
    sources = [
        {"path":"https://medium.com/@jh.baek.sd/chroma-vs-faiss-a-comparative-analysis-527a4f3c8fb", "type":"url"},
        {"path": "https://medium.com/analytics-vidhya/what-is-rnn-a157d903a88", "type": "url"},
        {"path": "https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04", "type": "url"},
        {"path":"https://vajiramandravi.com/upsc-exam/rafale-fighter-jet/", "type":"url"},
        {"path":"https://www.geeksforgeeks.org/implementing-the-adaboost-algorithm-from-scratch/", "type":"url"},
        {"path":"https://medium.com/civic-skunk-works/a-blinking-red-light-for-the-economy-fcf621189ed5", "type":"url"},
        {"path":"https://www.autosport.com/f1/news/how-does-a-formula-1-car-work-wings-diffusers-and-more-explained-4982275/4982275/", "type":"url"},
        {"path":"https://www.geeksforgeeks.org/model-context-protocol-mcp/", "type":"url"},
        {"path":"https://medium.com/@tahirbalarabe2/what-is-prompt-engineering-rag-cot-react-dsp-explained-0aa0a9bd0a90", "type":"url"},
        {"path":"https://www.geeksforgeeks.org/box-cox-transformation-using-python/", "type":"url"},
        {"path":"https://www.geeksforgeeks.org/bernoulli-distribution-in-business-statistics-mean-and-variance/", "type":"url"},
        {"path":"https://www.ibm.com/think/topics/fine-tuning", "type":"url"},
        {"path":"https://www.ibm.com/think/topics/ensemble-learning", "type":"url"},
        {"path":"https://www.geeksforgeeks.org/understanding-of-lstm-networks/", "type":"url"},

        {"path":"https://www.geeksforgeeks.org/what-is-crewai/", "type":"url"},


        ]

    documents = load_documents(sources)


    return documents

In [10]:
import re
import json
import nltk
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import wordnet

# Download necessary resources
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('punkt')
documents = process_sources()

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(tag):
    """Convert POS tag to wordnet format for lemmatizer"""
    if tag.startswith("J"):
        return wordnet.ADJ
    elif tag.startswith("V"):
        return wordnet.VERB
    elif tag.startswith("N"):
        return wordnet.NOUN
    elif tag.startswith("R"):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def preprocess_text(item):
    """Cleans and preprocesses AI article content"""
    source_url = item["source"]
    text = item["content"][1].lower()
    title=item["content"][0]
    tokens = word_tokenize(text)
    pos_tags = pos_tag(tokens)

    cleaned_tokens = [
        lemmatizer.lemmatize(word, get_wordnet_pos(tag))
        for word, tag in pos_tags
        if word not in stop_words and word.isalpha()
    ]

    cleaned_text = " ".join(cleaned_tokens)
    return {"source": source_url,"title":title, "cleaned_text": cleaned_text}

# Applying preprocessing in parallel
cleaned_data = []
with ThreadPoolExecutor(max_workers=6) as executor:
    results = list(tqdm(executor.map(preprocess_text, documents), total=len(documents), desc="Preprocessing AI Articles"))
    cleaned_data.extend(results)


with open("cleaned_ai_articles.json", "w", encoding="utf-8") as f:
    json.dump(cleaned_data, f, indent=2)

# Preview
print(cleaned_data[:2])


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Preprocessing AI Articles: 100%|██████████| 15/15 [00:02<00:00,  5.52it/s]

[{'source': 'https://medium.com/@jh.baek.sd/chroma-vs-faiss-a-comparative-analysis-527a4f3c8fb', 'title': 'Chroma vs FAISS: A Comparative Analysis | by ZIRU | Medium', 'cleaned_text': 'story chroma v faiss comparative analysis win unpack feature chroma faiss ziru follow min read aug share photo caspar camille rubin unsplash compare chroma faiss involves examine feature use case performance c hroma vector store embeddings database design make easy build ai application embeddings main feature include easy setup chroma design run machine instal chroma simple run pip install command local development chroma build run seamlessly local development make easy prototype ai application design ai workload chroma build handle modern ai workload make good fit application heavily utilize embeddings f aiss hand library develop facebook efficient similarity search clustering dense vector feature include efficiency faiss design efficient similarity search crucial application involve semantic search ope

In [11]:
def classify_ai_article(cleaned_text):
    """
    Classifies a cleaned text into ML/DL/GenAI/NLP/CV categories.
    Multi-label classification based on keyword matching.
    """
    categories = set()
    text = cleaned_text.lower()

    ml_keywords = ["regression", "classification", "supervised", "unsupervised", "adaboost","Random Forest", "ensemble", "svm"]
    dl_keywords = ["neural network", "cnn", "lstm", "rnn", "backpropagation", "deep learning"]
    genai_keywords = ["gpt", "llm","", "generative", "chatgpt",  "gemini","Anthropic", "prompt engineering","vector databases","chroma","faiss","pinecone","MCP"]
    nlp_keywords = ["nlp", "tokenization", "transformer", "bert", "language model", "attention"]
    cv_keywords = ["image", "vision", "cnn", "object detection", "segmentation"]


    for word in ml_keywords:
        if word in text:
            categories.add("ML")
            break
    for word in dl_keywords:
        if word in text:
            categories.add("DL")
            break
    for word in genai_keywords:
        if word in text:
            categories.add("GenAI")
            break
    for word in nlp_keywords:
        if word in text:
            categories.add("NLP")
            break
    for word in cv_keywords:
        if word in text:
            categories.add("CV")
            break

    if not categories:
        categories.add("Other")

    return list(categories)


In [12]:
import pandas as pd
for item in cleaned_data:
    item["categories"] = classify_ai_article(item["cleaned_text"])
cat_df = pd.DataFrame(cleaned_data)


(cat_df.head())


,source,title,cleaned_text,categories
0,https://medium.com/@jh.baek.sd/chroma-vs-faiss...,Chroma vs FAISS: A Comparative Analysis | by Z...,story chroma v faiss comparative analysis win ...,[GenAI]
1,https://medium.com/analytics-vidhya/what-is-rn...,Recurrent Neural Networks — Complete and In-de...,recurrent neural network complete tejas follow...,"[ML, GenAI, NLP, DL, CV]"
2,https://medium.com/inside-machine-learning/wha...,What is a Transformer?. An Introduction to Tra...,transformer maxime follow min read jan listen ...,"[ML, GenAI, NLP, DL, CV]"
3,https://vajiramandravi.com/upsc-exam/rafale-fi...,Rafale Fighter Jets - All about India's Rafale...,home rafale aircraft rafale fighter jet india ...,[GenAI]
4,https://www.geeksforgeeks.org/implementing-the...,Implementing the AdaBoost Algorithm From Scrat...,implement adaboost algorithm scratch last upda...,"[ML, GenAI]"


In [14]:
from google.colab import userdata
Gemini_api_key=userdata.get('Gemini_api_key')

import google.generativeai as genai
#generating abstractive summary using gemini as the system's motive is to assist students
# so keeping it in more human readable format

genai.configure(api_key=Gemini_api_key)

model = genai.GenerativeModel("gemini-1.5-pro")
import pandas as pd
import time
import re

summaries = []

for index, row in cat_df.iterrows():
    try:
        text = row["cleaned_text"][:15000]
        source = row["source"]

        prompt = f"""
        Summarize the following article in two ways:

        1. A **brief summary** in 2–3 lines.
        2. A **detailed summary** in 5–7 lines.

        Article:
        {text}
        """
        response = model.generate_content(prompt)
        output = response.text.strip()

        summaries.append({
            "source": source,
            "summary": output
        })

        time.sleep(1)

    except Exception as e:
        summaries.append({
            "source": row["source"],
            "summary": f"Error: {str(e)}"
        })

# Convert to DataFrame and save
summary_df = pd.DataFrame(summaries)

summary_df.head()


,source,summary
0,https://medium.com/@jh.baek.sd/chroma-vs-faiss...,**Brief Summary (2-3 lines):**\n\nThis article...
1,https://medium.com/analytics-vidhya/what-is-rn...,**Brief Summary:**\n\nThis article explores us...
2,https://medium.com/inside-machine-learning/wha...,"Error: ('Connection aborted.', RemoteDisconnec..."
3,https://vajiramandravi.com/upsc-exam/rafale-fi...,**Brief Summary:**\n\nThe Rafale is a versatil...
4,https://www.geeksforgeeks.org/implementing-the...,"Error: ('Connection aborted.', RemoteDisconnec..."


In [15]:

df = summary_df

# Function to split summary into brief and detailed parts
def split_summary(text):
    # Check if there's a detailed summary section
    if "**Detailed Summary" in text:
        parts = text.split("**Detailed Summary")
        brief_summary = parts[0].replace("**Brief Summary (2-3 lines):**", "").replace("**Brief Summary:**", "").strip()
        detailed_summary = parts[1].split(":**")[1].strip() if ":**" in parts[1] else parts[1].strip()
    else:
        # If no detailed summary, put everything in brief summary
        brief_summary = text.replace("**Brief Summary (2-3 lines):**", "").replace("**Brief Summary:**", "").strip()
        detailed_summary = ""

    return brief_summary, detailed_summary

# Apply function to create new columns
df[['Brief Summary', 'Detailed Summary']] = df['summary'].apply(lambda x: pd.Series(split_summary(x)))

# Select only the columns we want in the final dataframe
result_df = df[['source', 'Brief Summary', 'Detailed Summary']]




In [16]:
result_df.to_json("split_gemini_summaries.json", indent=2)

In [32]:
import math
import json

def chunk_document(text, chunk_size=500):
    """Splits text into fixed-size chunks."""
    return [text[i:i+chunk_size] for i in range(0, len(text), chunk_size)]

chunked_data = []

for _, row in cat_df.iterrows():
    chunks = chunk_document(row['cleaned_text'], chunk_size=500)
    for chunk in chunks:
        chunked_data.append({
            "title": row['title'],
            "source": row['source'],
            "category": row['categories'],
            "text_chunk": chunk
        })


with open("chunked_ai_articles.json", "w", encoding="utf-8") as f:
    json.dump(chunked_data, f, indent=2, ensure_ascii=False)


In [39]:
import numpy as np
import faiss
import json
import os
import time
import random
from tqdm import tqdm
from google import genai
from google.genai import types

# Configure API
client = genai.Client(api_key=Gemini_api_key)

# File paths
INDEX_PATH = "ai_articles_index.faiss"
METADATA_PATH = "ai_articles_metadata.json"

# Load existing FAISS index + metadata
def load_existing():
    if os.path.exists(INDEX_PATH):
        index = faiss.read_index(INDEX_PATH)
        with open(METADATA_PATH, 'r') as f:
            metadata = json.load(f)
        print(f"Loaded index with {index.ntotal} vectors.")
    else:
        index = None
        metadata = []
        print("No existing index found.")
    return index, metadata

# Save FAISS index + metadata
def save_all(index, metadata):
    faiss.write_index(index, INDEX_PATH)
    with open(METADATA_PATH, 'w') as f:
        json.dump(metadata, f, indent=2)
    print(f"Saved index with {index.ntotal} vectors.")

# Retry logic for Gemini embedding
def get_embedding(text, retries=5, delay=1):
    for attempt in range(retries):
        try:
            result = client.models.embed_content(
                model="gemini-embedding-exp-03-07",
                contents=text,
                config=types.EmbedContentConfig(task_type="RETRIEVAL_DOCUMENT")
            )
            return result.embeddings[0].values
        except Exception as e:
            if "429" in str(e) or "RESOURCE_EXHAUSTED" in str(e):
                wait = delay * (2 ** attempt) + random.uniform(0, 0.5)
                print(f"Rate limited. Waiting {wait:.1f}s...")
                time.sleep(wait)
            else:
                print(f"Error: {e}")
                if attempt == retries - 1:
                    raise e

# Main embedding function
def embed_chunks(chunks_json_path):
    with open(chunks_json_path, 'r') as f:
        chunks = json.load(f)
    print(f"Loaded {len(chunks)} text chunks.")

    index, metadata = load_existing()

    # Track already processed by (source + first 30 chars of text)
    processed_keys = {
        (m["source"], m["text_chunk"][:30]) for m in metadata
    }

    new_embeddings = []
    new_metadata = []
    dim = None  # To store embedding dimension

    for chunk in tqdm(chunks):
        key = (chunk["source"], chunk["text_chunk"][:30])
        if key in processed_keys:
            continue

        try:
            emb = get_embedding(chunk["text_chunk"])

            # Check if this is our first embedding or if dimensions match
            if dim is None:
                dim = len(emb)
            elif len(emb) != dim:
                print(f"Warning: Embedding dimension mismatch. Expected {dim}, got {len(emb)}. Skipping chunk.")
                continue

            new_embeddings.append(emb)
            new_metadata.append({
    "source": chunk["source"],
    "title": chunk.get("title", ""),
    "text_chunk": chunk["text_chunk"]
})


        except Exception as e:
            print(f"Failed to embed chunk: {e}")
            continue

    # Save to FAISS
    if new_embeddings:
        try:
            # Ensure consistent dimensions before creating array
            emb_array = np.array(new_embeddings, dtype=np.float32)

            if index is None:
                index = faiss.IndexFlatL2(emb_array.shape[1])

            index.add(emb_array)
            metadata.extend(new_metadata)
            save_all(index, metadata)
            print(f"Added {len(new_embeddings)} new vectors.")
        except ValueError as e:
            print(f"Error creating embedding array: {e}")
            print("Trying to save individually...")

            # Alternative approach: Add vectors one by one
            if index is None and dim:
                index = faiss.IndexFlatL2(dim)

            successful = 0
            for i, emb in enumerate(new_embeddings):
                try:
                    single_vector = np.array([emb], dtype=np.float32)
                    index.add(single_vector)
                    metadata.append(new_metadata[i])
                    successful += 1
                except Exception as e:
                    print(f"Failed to add vector {i}: {e}")

            if successful > 0:
                save_all(index, metadata)
                print(f"Added {successful} vectors individually.")
    else:
        print("No new embeddings added.")

# Call the embedding function
if __name__ == "__main__":
    embed_chunks("chunked_ai_articles.json")

Loaded 300 text chunks.
No existing index found.


 17%|█▋        | 50/300 [00:27<02:13,  1.88it/s]

Rate limited. Waiting 1.3s...


 17%|█▋        | 51/300 [00:29<03:51,  1.07it/s]

Rate limited. Waiting 1.2s...
Rate limited. Waiting 2.1s...
Rate limited. Waiting 4.2s...
Rate limited. Waiting 8.3s...
Rate limited. Waiting 16.1s...


...

Rate limited. Waiting 1.3s...
Rate limited. Waiting 2.3s...
Rate limited. Waiting 4.0s...
Rate limited. Waiting 8.2s...
Rate limited. Waiting 16.1s...


 86%|████████▌ | 258/300 [04:53<07:02, 10.07s/it]

Failed to embed chunk: object of type 'NoneType' has no len()


100%|██████████| 300/300 [05:16<00:00,  1.05s/it]

Saved index with 296 vectors.
Added 296 new vectors.


In [40]:
def search_similar_excerpts(query, index, metadata, top_k=5):
    """
    Search for similar excerpts using FAISS index.

    Args:
        query (str): The query text to search for
        index (faiss.Index): The FAISS index object
        metadata (list): List of metadata dictionaries for each vector
        top_k (int): Number of top results to return

    Returns:
        list: List of dictionaries containing text and source information
    """
    # Embed the query
    query_embedding = get_embedding(query, retries=2, delay=1)

    # Need to convert the embedding to a numpy array with correct shape and type
    query_array = np.array([query_embedding]).astype('float32')

    # Search in FAISS
    D, I = index.search(query_array, top_k)

    # Return results
    results = []
    for i, idx in enumerate(I[0]):
        if idx != -1 and idx < len(metadata):  # Check for valid index (-1 means no result found)
            result = {
                "title": metadata[idx].get('title', 'N/A'),
                "score": float(D[0][i]),
                "source": metadata[idx].get('source', 'N/A'),
                "text_chunk": metadata[idx].get('text_chunk', metadata[idx].get('text', 'N/A'))
            }
            results.append(result)

    return results


# Load existing index and metadata
def load_index_and_search(query, top_k=5):
    """Helper function to load index and search in one step"""
    import faiss
    import json

    # Load index and metadata
    INDEX_PATH = "ai_articles_index.faiss"
    METADATA_PATH = "ai_articles_metadata.json"

    index = faiss.read_index(INDEX_PATH)
    with open(METADATA_PATH, 'r') as f:
        metadata = json.load(f)

    # Perform search
    return search_similar_excerpts(query, index, metadata, top_k)

In [43]:
query = "What do you understand by encoders and decoders?"
results = load_index_and_search(query, top_k=5)

for i, r in enumerate(results, 1):
    print(f"{i} Title:{r['title']}\n Source: {r['source']}\n {r['text_chunk']}\n")

1 Title:What is a Transformer?. An Introduction to Transformers and… | by Maxime | Inside Machine learning | Medium
 Source: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
  take input sequence map high dimensional space vector abstract vector feed decoder turn output sequence output sequence another language symbol copy input etc imagine encoder decoder human translator speak two language first language mother tongue differ german french second language imaginary one common translate german french encoder convert german sentence language know namely imaginary language since decoder able read imaginary language translate language french together model consist encod

2 Title:💡 What is Prompt Engineering ?:: RAG, CoT, ReAct & DSP Explained | by Tahir | Medium
 Source: https://medium.com/@tahirbalarabe2/what-is-prompt-engineering-rag-cot-react-dsp-explained-0aa0a9bd0a90
 nstructured question

3 Title:What is a Transformer?. An Introduction to Transformers an

In [44]:
from google import genai
import json
import time
import re
import random
from typing import List, Dict, Any

def rerank_with_gemini(query: str, search_results: List[Dict[str, Any]],
                       client = client, model_name="gemini-1.5-pro",
                       top_k=3, max_retries=3):
    """
    Rerank search results using Gemini 1.5 Pro to improve relevancy.

    Args:
        query (str): The original user query
        search_results (list): List of initial search results (from FAISS)
        client: Gemini API client (if None, will use global client)
        model_name (str): Gemini model to use
        top_k (int): Number of results to return after reranking
        max_retries (int): Maximum number of retries on API failure

    Returns:
        list: Reranked search results
    """
    if not search_results:
        return []



    # Prepare content for Gemini
    passages = []
    for i, result in enumerate(search_results):
        passages.append(f"Passage {i+1}: {result['text_chunk']}")

    passages_text = "\n\n".join(passages)

    # Create the prompt for Gemini
    prompt = f"""
    You are a search reranker. I'll provide a query and several passages.
    Rerank the passages based on their relevance to the query.

    Query: {query}

    Passages:
    {passages_text}

    Instructions:
    1. Analyze each passage for relevance to the query
    2. Consider both semantic relevance and information value
    3. Provide a JSON response with the reranked passages
    4. Include a relevance score (1-10) and a brief explanation

    Return only valid JSON in this format:
    {{
        "reranked_results": [
            {{
                "passage_index": [original passage number],
                "relevance_score": [score between 1-10],
                "explanation": [brief explanation of relevance]
            }},
            ...
        ]
    }}
    """

    # Function to make API call with retry logic
    def call_gemini_with_retry(attempts=0):
        try:
            response = client.models.generate_content(
                model=model_name,
                contents=prompt,
                config=types.GenerateContentConfig(
                temperature=0.1,
                response_mime_type="application/json",
                max_output_tokens=2048
            ),
            )
            return response.text
        except Exception as e:
            if attempts < max_retries:
                wait_time = 2 ** attempts + random.uniform(0, 1)
                print(f"API error: {e}. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
                return call_gemini_with_retry(attempts + 1)
            else:
                print(f"Failed after {max_retries} attempts: {e}")
                # Fall back to original order if reranking fails
                return json.dumps({
                    "reranked_results": [
                        {"passage_index": i+1, "relevance_score": 5, "explanation": "Default ranking"}
                        for i in range(len(search_results))
                    ]
                })

    # Get reranking from Gemini
    response_text = call_gemini_with_retry()

    # Extract JSON from response (in case model outputs additional text)
    json_match = re.search(r'({.*})', response_text.replace('\n', ' '), re.DOTALL)
    if json_match:
        try:
            reranking_data = json.loads(json_match.group(1))
        except json.JSONDecodeError:
            print("Error parsing JSON from Gemini response")
            return search_results[:top_k]  # Return original order if parsing fails
    else:
        print("No JSON found in Gemini response")
        return search_results[:top_k]

    # Process reranking results
    reranked = []
    try:
        # Sort by relevance score in descending order
        sorted_rankings = sorted(
            reranking_data["reranked_results"],
            key=lambda x: x["relevance_score"],
            reverse=True
        )

        # Map back to original results
        for ranking in sorted_rankings[:top_k]:
            orig_idx = ranking["passage_index"] - 1  # Convert back to 0-based index
            if 0 <= orig_idx < len(search_results):
                result = search_results[orig_idx].copy()
                result["relevance_score"] = ranking["relevance_score"]
                result["explanation"] = ranking["explanation"]
                reranked.append(result)
    except (KeyError, IndexError) as e:
        print(f"Error processing reranking results: {e}")
        return search_results[:top_k]  # Return original order if processing fails

    return reranked

# Example usage
def search_and_rerank(query, top_k_initial=10, top_k_final=5):
    """
    Perform vector search and rerank results using Gemini.

    Args:
        query (str): The user's query
        top_k_initial (int): Number of initial results to get from FAISS
        top_k_final (int): Number of results to return after reranking

    Returns:
        list: Reranked search results
    """
    # First get initial results from FAISS (get more than we need for reranking)
    initial_results = load_index_and_search(query, top_k=top_k_initial)

    # Then rerank using Gemini
    reranked_results = rerank_with_gemini(query, initial_results, top_k=top_k_final)

    return reranked_results

In [46]:
query = "What do you understand by encoders and decoders?"
results = search_and_rerank(query, top_k_initial=10, top_k_final=5)

# now here the results are based on semantic meaning rather than simailarity scores
for i, r in enumerate(results, 1):
    print(f"{i} Title:{r['title']}\n Source: {r['source']}\n {r['text_chunk']}\n")

1 Title:What is a Transformer?. An Introduction to Transformers and… | by Maxime | Inside Machine learning | Medium
 Source: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
 ll architecture neural net transform give sequence element sequence word sentence another sequence well might surprise consider name model particularly good translation sequence word one language transform sequence different word another language popular choice type model lstm model data lstm module give meaning sequence remember forget part find important unimportant sentence example since order word crucial understand sentence lstm natural choice type data model consist encoder decoder encoder

2 Title:What is a Transformer?. An Introduction to Transformers and… | by Maxime | Inside Machine learning | Medium
 Source: https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
  paper prove architecture without rnn recurrent neural network improve result translation 

 here the differences in results is clearly visible


In [48]:
from google import genai
import numpy as np
import json
import time
import random
from typing import List, Dict, Any

def answer_with_rag(query: str,
                   client=client,
                   model_name="gemini-1.5-pro",
                   initial_results=10,
                   max_context_passages=5,
                   max_retries=3):
    """
    Complete RAG (Retrieval-Augmented Generation) system that:
    1. Retrieves relevant passages using vector search
    2. Reranks them using Gemini
    3. Generates a comprehensive answer based on the retrieved information

    Args:
        query (str): The user's question
        client: Gemini API client (if None, will use global client)
        model_name (str): Gemini model to use
        initial_results (int): Number of initial results to get from vector search
        max_context_passages (int): Maximum passages to include in the context
        max_retries (int): Maximum number of retries on API failure

    Returns:
        dict: {
            "answer": LLM-generated answer to the query,
            "sources": List of sources with metadata and excerpts
        }
    """


    # Step 1: Get initial search results from FAISS
    search_results = load_index_and_search(query, top_k=initial_results)

    if not search_results:
        return {
            "answer": "I couldn't find any relevant information to answer your question.",
            "sources": []
        }

    # Step 2: Rerank the results using Gemini
    reranked_results = rerank_with_gemini(
        query=query,
        search_results=search_results,
        client=client,
        model_name=model_name,
        top_k=max_context_passages,
        max_retries=max_retries
    )

    # Step 3: Format the context information from reranked results
    context_passages = []
    sources = []

    for i, result in enumerate(reranked_results):
        # Get the text chunk for context
        text_chunk = result['text_chunk']
        context_passages.append(f"[{i+1}] {text_chunk}")

        # Store comprehensive source information including the excerpt
        sources.append({
            "title": result.get("title", "Unknown title"),
            "source": result.get("source", "Unknown source"),
            "relevance_score": result.get("relevance_score", None),
            "explanation": result.get("explanation", None),
            "excerpt": text_chunk  # Store the full text excerpt
        })

    context_text = "\n\n".join(context_passages)

    # Step 4: Generate a comprehensive answer using Gemini
    prompt = f"""
    You are an expert AI assistant that provides accurate, informative answers based on retrieved information.

    USER QUERY: {query}

    RELEVANT INFORMATION:
    {context_text}

    INSTRUCTIONS:
    1. Answer the user's query thoroughly based on the provided information
    2. Cite your sources using the corresponding numbers [1], [2], etc.
    3. If the retrieved information doesn't fully answer the query, acknowledge the limitations
    4. Present the information in a clear, well-structured format
    5. Be objective and accurate - only use information from the provided passages
    6. If the query cannot be answered with the retrieved information, state that clearly

    YOUR ANSWER:
    """

    # Function to make API call with retry logic
    def generate_answer_with_retry(attempts=0):
        try:
            response = client.models.generate_content(
                model=model_name,
                contents=prompt,
                config=types.GenerateContentConfig(
                temperature=0.1,
                response_mime_type="application/json",
                max_output_tokens=2048
            ),
            )
            return response.text
        except Exception as e:
            if attempts < max_retries:
                wait_time = 2 ** attempts + random.uniform(0, 1)
                print(f"API error: {e}. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
                return generate_answer_with_retry(attempts + 1)
            else:
                print(f"Failed after {max_retries} attempts: {e}")
                return "I encountered an error while trying to generate an answer. Please try again later."

    # Generate the answer
    answer = generate_answer_with_retry()

    return {
        "answer": answer,
        "sources": sources
           }

# Enhanced display function with excerpts
def display_rag_results(results):
    """
    Display the RAG results in a readable format including source excerpts

    Args:
        results (dict): Results from answer_with_rag function
    """
    print("=" * 100)
    print("ANSWER:")
    print("=" * 100)
    print(results["answer"])
    print("\n" + "=" * 100)
    print("SOURCES WITH EXCERPTS:")
    print("=" * 100)

    for i, source in enumerate(results["sources"]):
        print(f"[{i+1}] {source['source']}")
        print(f" Title: {source['title']}")
        if source.get("relevance_score"):
            print(f"    Relevance: {source['relevance_score']}/10")
        if source.get("explanation"):
            print(f"    Explanation: {source['explanation']}")

        # Display the excerpt
        print("\n    EXCERPT:")
        print(f"    {'-' * 80}")
        # Format the excerpt with proper indentation for readability
        excerpt_lines = source['excerpt'].split('\n')
        for line in excerpt_lines:
            print(f"    {line}")
        print(f"    {'-' * 80}\n")

    print("=" * 100)

# Example usage:
answer = answer_with_rag("What is masking in transformer?")
display_rag_results(answer)



ANSWER:
{"answer": "Masking in Transformers, specifically within the context of training, involves preventing the model from \"cheating\" by seeing the target word it's supposed to predict.  The provided texts describe the general architecture and mechanisms of the Transformer model, including positional encoding and the encoder-decoder structure [1, 2, 3, 4, 5]. However, they don't explicitly detail the concept of masking.  The text mentions shifting the decoder input to the right by one position to prevent the model from simply copying the input [5]. This shifting is related to the concept of masking, but it doesn't fully explain how masking works in the attention mechanism itself.  Therefore, while the provided texts offer some clues, they don't contain enough information to thoroughly explain masking in Transformers.", "sources": [1, 2, 3, 4, 5]}

SOURCES WITH EXCERPTS:
[1] https://medium.com/inside-machine-learning/what-is-a-transformer-d07dd1fbec04
 Title: What is a Transformer?.

In [49]:
from google import genai
import numpy as np
import json
import time
import random
from typing import List, Dict, Any

def answer_with_rag_cot(query: str,
                    client=client,
                    model_name="gemini-1.5-pro",
                    initial_results=10,
                    max_context_passages=5,
                    max_retries=3,
                    include_cot=True):
    """
    RAG system with Chain-of-Thought reasoning:
    1. Retrieves relevant passages using vector search
    2. Reranks them using Gemini
    3. Applies Chain-of-Thought reasoning
    4. Generates a comprehensive answer based on the retrieved information

    Args:
        query (str): The user's question
        client: Gemini API client (if None, will use global client)
        model_name (str): Gemini model to use
        initial_results (int): Number of initial results to get from vector search
        max_context_passages (int): Maximum passages to include in the context
        max_retries (int): Maximum number of retries on API failure
        include_cot (bool): Whether to include the chain-of-thought reasoning in output

    Returns:
        dict: {
            "answer": LLM-generated answer to the query,
            "sources": List of sources with metadata and excerpts,
            "cot_reasoning": Chain-of-thought reasoning process (if include_cot=True)
        }
    """


    # Step 1: Get initial search results from FAISS
    search_results = load_index_and_search(query, top_k=initial_results)

    if not search_results:
        return {
            "answer": "I couldn't find any relevant information to answer your question.",
            "sources": [],
            "cot_reasoning": "No relevant information found."
        }

    # Step 2: Rerank the results using Gemini
    reranked_results = rerank_with_gemini(
        query=query,
        search_results=search_results,
        client=client,
        model_name=model_name,
        top_k=max_context_passages,
        max_retries=max_retries
    )

    # Step 3: Format the context information from reranked results
    context_passages = []
    sources = []

    for i, result in enumerate(reranked_results):
        # Get the text chunk for context
        text_chunk = result['text_chunk']
        context_passages.append(f"[{i+1}] {text_chunk}")

        # Store comprehensive source information including the excerpt
        sources.append({
            "title": result.get("title", "Unknown title"),
            "source": result.get("source", "Unknown source"),
            "relevance_score": result.get("relevance_score", None),
            "explanation": result.get("explanation", None),
            "excerpt": text_chunk  # Store the full text excerpt
        })

    context_text = "\n\n".join(context_passages)

    # Step 4: Apply Chain-of-Thought reasoning
    cot_prompt = f"""
    You are an expert AI assistant that uses Chain-of-Thought reasoning to analyze information and answer questions.

    USER QUERY: {query}

    RELEVANT INFORMATION:
    {context_text}

    INSTRUCTIONS:
    Perform a detailed Chain-of-Thought reasoning process to answer the query. Follow these steps:

    Step 1: Analyze each retrieved context
    - Summarize key information from each source
    - Evaluate the relevance and reliability of each source
    - Identify any contradictions or gaps between sources

    Step 2: Identify key concepts relevant to the query
    - Extract the main concepts, facts, and relationships
    - Determine which information is most pertinent to answering the query
    - Connect related pieces of information across different sources

    Step 3: Synthesize a comprehensive answer
    - Integrate information from all relevant sources
    - Address the query directly and completely
    - Acknowledge any limitations or uncertainties

    Format your response as a detailed reasoning process through these steps. Be explicit about your reasoning.
    Use exact citations to the source materials using [1], [2], etc.

    YOUR CHAIN-OF-THOUGHT REASONING:
    """

    # Function to make API call with retry logic
    def generate_cot_with_retry(attempts=0):
        try:
            response = client.models.generate_content(
                model=model_name,
                contents=cot_prompt,
                config=types.GenerateContentConfig(
                temperature=0.1,
                response_mime_type="application/json",
                max_output_tokens=2048
            ),
            )
            return response.text
        except Exception as e:
            if attempts < max_retries:
                wait_time = 2 ** attempts + random.uniform(0, 1)
                print(f"API error: {e}. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
                return generate_cot_with_retry(attempts + 1)
            else:
                print(f"Failed after {max_retries} attempts: {e}")
                return "I encountered an error while trying to generate the chain-of-thought reasoning."

    # Generate the chain-of-thought reasoning
    cot_reasoning = generate_cot_with_retry()

    # Step 5: Generate final answer based on CoT reasoning
    answer_prompt = f"""
    You are an expert AI assistant that provides accurate, informative answers based on retrieved information.

    USER QUERY: {query}

    You've already performed a detailed chain-of-thought reasoning process:

    {cot_reasoning}

    Now, provide a clear, concise, and comprehensive answer to the user's query based on your reasoning.
    - Cite your sources using the corresponding numbers [1], [2], etc.
    - Focus on being accurate, informative, and helpful
    - Present the information in a well-structured format
    - Be direct and to the point

    YOUR FINAL ANSWER:
    """

    def generate_answer_with_retry(attempts=0):
        try:
            response = client.models.generate_content(
                model=model_name,
                contents=answer_prompt,
                config=types.GenerateContentConfig(
                temperature=0.1,
                response_mime_type="application/json",
                max_output_tokens=2048
            ),
            )
            return response.text
        except Exception as e:
            if attempts < max_retries:
                wait_time = 2 ** attempts + random.uniform(0, 1)
                print(f"API error: {e}. Retrying in {wait_time:.1f} seconds...")
                time.sleep(wait_time)
                return generate_answer_with_retry(attempts + 1)
            else:
                print(f"Failed after {max_retries} attempts: {e}")
                return "I encountered an error while trying to generate an answer. Please try again later."

    # Generate the final answer
    answer = generate_answer_with_retry()

    result = {
        "answer": answer,
        "sources": sources
           }

    # Include CoT reasoning if requested
    if include_cot:
        result["cot_reasoning"] = cot_reasoning

    return result

# Enhanced display function with chain-of-thought reasoning
def display_rag_cot_results(results):
    """
    Display the RAG results with Chain-of-Thought reasoning in a readable format

    Args:
        results (dict): Results from answer_with_rag_cot function
    """
    print("=" * 100)
    print("ANSWER:")
    print("=" * 100)
    print(results["answer"])

    if "cot_reasoning" in results:
        print("\n" + "=" * 100)
        print("CHAIN-OF-THOUGHT REASONING:")
        print("=" * 100)
        print(results["cot_reasoning"])

    print("\n" + "=" * 100)
    print("SOURCES WITH EXCERPTS:")
    print("=" * 100)

    for i, source in enumerate(results["sources"]):
        print(f"[{i+1}] {source['source']}")
        print(f" Title: {source['title']}")
        if source.get("relevance_score"):
            print(f"    Relevance: {source['relevance_score']}/10")
        if source.get("explanation"):
            print(f"    Explanation: {source['explanation']}")

        # Display the excerpt
        print("\n    EXCERPT:")
        print(f"    {'-' * 80}")
        # Format the excerpt with proper indentation for readability
        excerpt_lines = source['excerpt'].split('\n')
        for line in excerpt_lines:
            print(f"    {line}")
        print(f"    {'-' * 80}\n")

    print("=" * 100)

results = answer_with_rag_cot("What are the advantages and limitations of transformer models?")
display_rag_cot_results(results)

ANSWER:
{"Advantages": ["Parallelizable architecture enabling efficient training and inference, especially with large datasets [2, 3]", "Excellent at capturing long-range dependencies in sequences due to the attention mechanism [2, 4]", "Reduced need for manual feature engineering [1]"], "Limitations": ["Computationally intensive training, particularly for very large datasets [1]", "Specific limitations beyond training complexity are not detailed in the provided context"], "Overall": "Transformer models offer significant advantages over traditional RNNs like LSTMs and GRUs, especially in terms of efficiency and ability to handle long sequences. However, training can be resource-intensive, and further research is needed to fully explore their limitations."}

CHAIN-OF-THOUGHT REASONING:
{"Step 1: Analyze each retrieved context": [{"Source": "[1]", "Summary": "RNNs, LSTMs, and GRUs have limitations in training complexity and potential effectiveness issues.  Manual feature engineering is r